In [ ]:
import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as T
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset

In [ ]:
import torch
import torchvision
import torchvision.transforms as T

# 1) CIFAR-10 (clean) transforms & loader
clean_transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])

# Clean CIFAR-10 test set
cifar10_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
cifar10_test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=128, shuffle=False, num_workers=2)


In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import ConcatDataset, Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms

# Load corruption file
def load_noise(noise_type, severity=3, path=''):
    # Load corruption file
    corruption_file = np.load(os.path.join(path, f"{noise_type}.npy"))

    # Load CIFAR-10 clean test labels
    cifar10_test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)
    labels = torch.tensor(cifar10_test.targets)  # 10,000 labels

    # Pick only the 10k images for the chosen severity
    start_idx = (severity - 1) * 10000
    end_idx = severity * 10000

    images = corruption_file[start_idx:end_idx]  # (10000, 32, 32, 3)
    images = torch.tensor(images).permute(0, 3, 1, 2).float() / 255.0  # (10000, 3, 32, 32)

    # Labels match now: 10k images <-> 10k labels
    dataset = torch.utils.data.TensorDataset(images, labels)
    loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=False, num_workers=0)

    return loader, dataset

def load_all_noises(corruptions, severity=3, base_path=''):
    ds_list = []
    for c in corruptions:
        ds_list.append(load_noise(c, severity, base_path)[1])
    return ConcatDataset(ds_list)


In [ ]:
noise_types = ["gaussian_noise","motion_blur","jpeg_compression","shot_noise","fog"]
noise = load_all_noises(noise_types)

KeyboardInterrupt: 

In [ ]:
noise.datasets[0][0]

(tensor([[[0.6314, 0.6667, 0.6392,  ..., 0.5373, 0.5176, 0.3961],
          [0.4588, 0.5843, 0.7569,  ..., 0.4980, 0.5294, 0.3529],
          [0.5412, 0.4431, 0.7725,  ..., 0.5255, 0.4745, 0.6431],
          ...,
          [0.0431, 0.2667, 0.0431,  ..., 0.1569, 0.0275, 0.0000],
          [0.0000, 0.3294, 0.0667,  ..., 0.1882, 0.1490, 0.0196],
          [0.2235, 0.4392, 0.1961,  ..., 0.0235, 0.0549, 0.0000]],
 
         [[0.4588, 0.5725, 0.3020,  ..., 0.3686, 0.2392, 0.3529],
          [0.4157, 0.3490, 0.4980,  ..., 0.3922, 0.2078, 0.2863],
          [0.4275, 0.3922, 0.4000,  ..., 0.2824, 0.4706, 0.2863],
          ...,
          [0.6078, 0.5098, 0.3569,  ..., 0.4510, 0.2627, 0.3373],
          [0.5333, 0.4510, 0.4157,  ..., 0.3176, 0.3098, 0.1333],
          [0.4941, 0.4627, 0.3961,  ..., 0.1412, 0.3451, 0.3294]],
 
         [[0.1020, 0.1451, 0.1804,  ..., 0.1569, 0.1686, 0.2078],
          [0.0706, 0.3020, 0.2000,  ..., 0.0000, 0.0706, 0.1176],
          [0.2941, 0.1294, 0.2275,  ...,

In [ ]:
# clean training set (with transform)
train_clean = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=clean_transform,
    target_transform=lambda y: torch.tensor(y)
)


# combined noise dataset (severity=3)
noises_ds = load_all_noises(noise_types, severity=3)

# merge clean + all noises, then wrap *that* in one DataLoader
mixed_ds = ConcatDataset([train_clean, noises_ds])
train_loader = DataLoader(
    mixed_ds,
    batch_size=128,
    shuffle=True,
    num_workers=0
)

In [ ]:
train_loader.dataset.cumulative_sizes

[50000, 100000]

In [ ]:
def evaluate_accuracy(model, loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for x,y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            correct += (preds==y).sum().item()
            total += y.size(0)
    return correct/total


baseline_errors = {
    "gaussian_noise":0.85, "motion_blur":0.90,
    "jpeg_compression":0.89, "shot_noise":0.88,
    "fog":0.93
}

def compute_mce(model, corruptions, severities):
    errs = []
    for c in corruptions:
        for s in severities:
            loader, dataset = load_noise(c, s)
            err = 1 - evaluate_accuracy(model, loader)
            errs.append(err / baseline_errors[c])
    return np.mean(errs)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True).to(device)
model.fc = nn.Linear(model.fc.in_features, 10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15], gamma=0.1)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# 7) Training loop with in-epoch evaluation & per-epoch checkpoints
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Average training loss
    avg_loss = running_loss / len(train_loader)

    # --- In-loop evaluation ---
    clean_acc = evaluate_accuracy(model, clean_loader)
    mce_seen = compute_mce(model, seen, severities=[1,3,5])

    print(
        f"Epoch {epoch+1:2d} | "
        f"Train Loss: {avg_loss:.4f} | "
        f"Clean Acc: {clean_acc*100:5.2f}% | "
        f"mCE (seen): {mce_seen:.3f}"
    )

    # --- Checkpoint ---
    ckpt_path = f"checkpoint_epoch{epoch+1:02d}.pt"
    torch.save(model.state_dict(), ckpt_path)
    print(f" Saved {ckpt_path}")

# 8) Final checkpoint (optional)
torch.save(model.state_dict(), "checkpoint_final.pt")
print("Saved checkpoint_final.pt")


Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 178, in close
    self._clo

Epoch  1 | Train Loss: 0.3314 | Clean Acc: 91.49% | mCE (seen): 0.101
 Saved checkpoint_epoch01.pt
Epoch  2 | Train Loss: 0.2390 | Clean Acc: 93.61% | mCE (seen): 0.056
 Saved checkpoint_epoch02.pt
Epoch  3 | Train Loss: 0.1843 | Clean Acc: 95.00% | mCE (seen): 0.058
 Saved checkpoint_epoch03.pt
Epoch  4 | Train Loss: 0.1419 | Clean Acc: 95.81% | mCE (seen): 0.049
 Saved checkpoint_epoch04.pt
Epoch  5 | Train Loss: 0.1140 | Clean Acc: 95.43% | mCE (seen): 0.051
 Saved checkpoint_epoch05.pt
Epoch  6 | Train Loss: 0.0898 | Clean Acc: 96.00% | mCE (seen): 0.046
 Saved checkpoint_epoch06.pt
Epoch  7 | Train Loss: 0.0776 | Clean Acc: 96.18% | mCE (seen): 0.038
 Saved checkpoint_epoch07.pt
Epoch  8 | Train Loss: 0.0674 | Clean Acc: 96.46% | mCE (seen): 0.024
 Saved checkpoint_epoch08.pt
Epoch  9 | Train Loss: 0.0564 | Clean Acc: 96.40% | mCE (seen): 0.024
 Saved checkpoint_epoch09.pt
Epoch 10 | Train Loss: 0.0491 | Clean Acc: 97.21% | mCE (seen): 0.026
 Saved checkpoint_epoch10.pt


KeyboardInterrupt: 

In [ ]:
# PEFT

# Freezing last layer doesnt help.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
for p in model.parameters(): p.requires_grad = False
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15], gamma=0.1)



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# 7) Training loop with in-epoch evaluation & per-epoch checkpoints
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Average training loss
    avg_loss = running_loss / len(train_loader)

    # --- In-loop evaluation ---
    clean_acc = evaluate_accuracy(model, clean_loader)
    mce_seen = compute_mce(model, seen, severities=[1,3,5])

    print(
        f"Epoch {epoch+1:2d} | "
        f"Train Loss: {avg_loss:.4f} | "
        f"Clean Acc: {clean_acc*100:5.2f}% | "
        f"mCE (seen): {mce_seen:.3f}"
    )

    # --- Checkpoint ---
    ckpt_path = f"checkpoint_epoch{epoch+1:02d}.pt"
    torch.save(model.state_dict(), ckpt_path)
    print(f" Saved {ckpt_path}")

clean_acc_pe = evaluate(model, clean_loader)
mce_pe = compute_mce(model, seen, [1,3,5])
print(f"Clean acc (PE): {clean_acc_pe:.3f}, mCE (PE): {mce_pe:.3f}")
print(f"Param-efficiency ratio: {(mce_seen-mce_pe)/sum(p.numel() for p in model.fc.parameters()):.3e}")

Epoch  1 | Train Loss: 1.9119 | Clean Acc: 42.33% | mCE (seen): 0.789
 Saved checkpoint_epoch01.pt
Epoch  2 | Train Loss: 1.8779 | Clean Acc: 42.61% | mCE (seen): 0.790
 Saved checkpoint_epoch02.pt
Epoch  3 | Train Loss: 1.8799 | Clean Acc: 42.43% | mCE (seen): 0.776
 Saved checkpoint_epoch03.pt
Epoch  4 | Train Loss: 1.8830 | Clean Acc: 43.23% | mCE (seen): 0.768
 Saved checkpoint_epoch04.pt
Epoch  5 | Train Loss: 1.8789 | Clean Acc: 42.58% | mCE (seen): 0.775
 Saved checkpoint_epoch05.pt
Epoch  6 | Train Loss: 1.8807 | Clean Acc: 43.23% | mCE (seen): 0.791
 Saved checkpoint_epoch06.pt


KeyboardInterrupt: 

In [ ]:
# Training with Unfreeze the last ResNet block (layer4) + BatchNorm
model = models.resnet18(pretrained=True).to(device)

# 2) Freeze everything
for p in model.parameters():
    p.requires_grad = False

# 3) Unfreeze layer4 and its BNs
for name, m in model.named_modules():
    if name.startswith("layer4") or isinstance(m, nn.BatchNorm2d):
        for p in m.parameters():
            p.requires_grad = True

# 4) Always unfreeze the classifier
model.fc = nn.Linear(model.fc.in_features, 10).to(device)

# 5) Collect only the trainable params
trainable = [p for p in model.parameters() if p.requires_grad]
print("Trainable params:", sum(p.numel() for p in trainable))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15], gamma=0.1)

Trainable params: 8403338


In [ ]:
# 7) Training loop with in-epoch evaluation & per-epoch checkpoints
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Average training loss
    avg_loss = running_loss / len(train_loader)

    # --- In-loop evaluation ---
    clean_acc = evaluate_accuracy(model, clean_loader)
    mce_seen = compute_mce(model, seen, severities=[1,3,5])

    print(
        f"Epoch {epoch+1:2d} | "
        f"Train Loss: {avg_loss:.4f} | "
        f"Clean Acc: {clean_acc*100:5.2f}% | "
        f"mCE (seen): {mce_seen:.3f}"
    )

    # --- Checkpoint ---
    ckpt_path = f"checkpoint_epoch{epoch+1:02d}.pt"
    torch.save(model.state_dict(), ckpt_path)
    print(f" Saved {ckpt_path}")

clean_acc_pe = evaluate(model, clean_loader)
mce_pe = compute_mce(model, seen, [1,3,5])
print(f"Clean acc (PE): {clean_acc_pe:.3f}, mCE (PE): {mce_pe:.3f}")
print(f"Param-efficiency ratio: {(mce_seen-mce_pe)/sum(p.numel() for p in model.fc.parameters()):.3e}")

Epoch  1 | Train Loss: 1.0505 | Clean Acc: 74.54% | mCE (seen): 0.287
 Saved checkpoint_epoch01.pt
Epoch  2 | Train Loss: 0.6856 | Clean Acc: 80.15% | mCE (seen): 0.217
 Saved checkpoint_epoch02.pt
Epoch  3 | Train Loss: 0.5165 | Clean Acc: 81.09% | mCE (seen): 0.147
 Saved checkpoint_epoch03.pt
Epoch  4 | Train Loss: 0.3979 | Clean Acc: 84.39% | mCE (seen): 0.092
 Saved checkpoint_epoch04.pt
Epoch  5 | Train Loss: 0.3286 | Clean Acc: 85.43% | mCE (seen): 0.090
 Saved checkpoint_epoch05.pt
Epoch  6 | Train Loss: 0.2626 | Clean Acc: 86.63% | mCE (seen): 0.075
 Saved checkpoint_epoch06.pt
Epoch  7 | Train Loss: 0.2153 | Clean Acc: 86.04% | mCE (seen): 0.057
 Saved checkpoint_epoch07.pt
Epoch  8 | Train Loss: 0.1812 | Clean Acc: 87.88% | mCE (seen): 0.051
 Saved checkpoint_epoch08.pt
Epoch  9 | Train Loss: 0.1548 | Clean Acc: 86.76% | mCE (seen): 0.056
 Saved checkpoint_epoch09.pt
Epoch 10 | Train Loss: 0.1325 | Clean Acc: 87.60% | mCE (seen): 0.053
 Saved checkpoint_epoch10.pt
Epoch 11 |

KeyboardInterrupt: 